In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Peptic_ulcer_disease"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Peptic_ulcer_disease/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Peptic_ulcer_disease/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Peptic_ulcer_disease/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Peptic_ulcer_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Find the directory corresponding to Stomach Cancer (related to Peptic Ulcer Disease)
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Find the directory that matches our trait: Peptic_ulcer_disease
# Peptic ulcers commonly occur in the stomach, so Stomach Cancer (STAD) is the most relevant dataset
stomach_dirs = [dir_name for dir_name in tcga_dirs if "stomach" in dir_name.lower()]

if not stomach_dirs:
    print(f"No matching directory found for trait: {trait}")
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
else:
    # Select the most relevant directory
    selected_dir = stomach_dirs[0]  # Should be 'TCGA_Stomach_Cancer_(STAD)'
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'antireflux_treatment', 'antireflux_treatment_type', 'barretts_esophagus', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'city_of_procurement', 'country_of_procurement', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'family_history_of_stomach_cancer', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'gender', 'h_pylori_infection', 'histological_t

### Step 2: Find Candidate Demographic Features

In [3]:
# Identifying candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# First, let's explore the directory structure to find the cohort directory
import os

# List all subdirectories in the TCGA root directory
cohorts = [d for d in os.listdir(tcga_root_dir) if os.path.isdir(os.path.join(tcga_root_dir, d))]
print(f"Available cohorts: {cohorts}")

# For TCGA, we need to find a cohort that might contain relevant files
clinical_df = None
age_preview = {}
gender_preview = {}

for cohort in cohorts:
    cohort_dir = os.path.join(tcga_root_dir, cohort)
    try:
        # Try to get clinical and genetic files for this cohort
        files = os.listdir(cohort_dir)
        clinical_files = [f for f in files if 'clinicalmatrix' in f.lower()]
        
        if clinical_files:
            print(f"Found clinical files in cohort: {cohort}")
            clinical_file_path = os.path.join(cohort_dir, clinical_files[0])
            clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
            break
    except:
        continue

if clinical_df is not None:
    # Preview age columns
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview[col] = clinical_df[col].head(5).tolist()
    
    # Preview gender columns
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview[col] = clinical_df[col].head(5).tolist()
    
    print("Age columns preview:")
    print(age_preview)
    print("\nGender columns preview:")
    print(gender_preview)
else:
    print("No suitable clinical data files found in any cohort")


Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_(UCEC)', 'TCGA_Esophageal_Cancer_(ESCA)',

### Step 3: Select Demographic Features

In [4]:
# Analyze age columns
age_col = "age_at_initial_pathologic_diagnosis"  # This column directly provides age values
# "days_to_birth" could be used but would require conversion from days to years

# Analyze gender columns 
gender_col = "gender"  # This column directly provides gender information

# Print chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Define the cohort directory
selected_dir = 'TCGA_Stomach_Cancer_(STAD)'  # Corrected to use Stomach Cancer dataset
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable instead of hardcoding
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for stomach cancer patients, which is relevant for peptic ulcer disease analysis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 580 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (450, 19851) (samples x features)


After handling missing values, data shape: (450, 19851)
For the feature 'Peptic_ulcer_disease', the least common label is '0' with 35 occurrences. This represents 7.78% of the dataset.
The distribution of the feature 'Peptic_ulcer_disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.0
  50% (Median): 67.0
  75%: 73.0
Min: 30.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 159 occurrences. This represents 35.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved usable linked data to ../../output/preprocess/Peptic_ulcer_disease/TCGA.csv
